In [1]:
import pandas as pd

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.4/logging/__init__.py", line 982, in emit
    self.flush()
  File "/usr/lib/python3.4/logging/__init__.py", line 962, in flush
    self.stream.flush()
OSError: [Errno 5] Input/output error
Call stack:
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.4/site-packages/IPython/kernel/zmq/kernelapp.py", line 469, in main
    app.start()
  File "/usr/lib/python3.4/site-packages/IPython/kernel/zmq/kernelapp.py", line 459, in start
    ioloop.IOLoop.instance().start()
  File "/usr/lib/python3.4/site-packages/zmq/eventloop/ioloop.py", line 151, in start
    super(ZMQIOLoop, self).start()
  File "/usr/lib/python3.4/site-packages/tornado/ioloop.py", line 837, in start
    handler_func(fd_obj, events)
  File "/usr/lib/python3.4/site-packages/tornado/stack_context.py", line 275, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/lib/python3.4/site-packages/zmq/eventloop/zmqstream.py", line 433,

In [3]:
sp500_companies = pd.DataFrame.from_csv("constituents.csv") ##used a script from the internet that scrapes spindices
sp500_companies['Name']=sp500_companies['Name'].str.replace('Co\.|Inc|\.|\s\s|\sCo$|\sCorp$','') #remove the titles such as Corp and Inc

In [4]:
company_names = list(sp500_companies['Name']) #create a list for the company names

In [5]:
import tweepy
from tweepy import StreamListener
from twitterSentiment import models
from datetime import datetime
from pytz import timezone
import json, time, sys

tweetsmax = 10
tweetscount = 0

def timeupdate(twitterdate):
        # method to return a django-supported time from twitter-based time entry
        # input comes in the following fashion: Tue Jul 02 14:33:59 +0000 2013
        # return 2013-06-18 18:23:22-04:00
        central = timezone('US/Central')
        return central.localize(datetime.strptime(twitterdate, '%a %b %d %H:%M:%S +0000 %Y'))
    
class TwitterListener(StreamListener):

    
    
    #ideas from http://digitalfoo.net/posts/using-python-and-tweepy-to-scrape-streaming-tweets-into-mongodb/
    #and http://nbviewer.ipython.org/github/alexhanna/hse-twitter/blob/master/docs/Collecting%20Twitter%20data%20from%20the%20API%20with%20Python.ipynb
    
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
   
    def on_data(self, data):
        global tweetsmax
        global tweetscount
        print (tweetsmax)
        print (tweetscount)
        if (tweetsmax == tweetscount):
            tweetscount=0
            return False
        else:
            tweetscount = tweetscount+1
        try:
            tweet = json.loads(data) #convert twitter stream in json into Python dictionary
            if isinstance(tweet, dict):
                if tweet['user']['lang'] != 'en':
                    return
                else:
                    print ("tweet: ", tweet['text'])
                    TwitterDatabase(tweet)
        except:
            print ("Error in Twitter listener. Error message:", sys.exc_info())
        return
    def on_limit(self, track):
        print(">> limit")
        return 

    def on_error(self, status_code):
        print(">>> error: ", str(status_code) + "\n")
        return 

    def on_timeout(self):
        print(">>> timeout Sleeping for 60 seconds...\n")
        time.sleep(60)
        return 

  
def TwitterDatabase(tweet):
    ## take Twitter data in jsonformat and insert into the database
    try:
        aTweet = models.TwitterText(twitter_user_id=tweet['user']['id'], 
                                        twitter_user_name=tweet['user']['screen_name'],
                                        twitter_text=tweet['text'], 
                                        twitter_text_id=tweet['id'], 
                                        twitter_text_timestamp=timeupdate(tweet['created_at']), 
                                        twitter_text_keyword=keywords)
        aTweet.save()
    except:
        print ("Error in Django insert tweet. error message:", sys.exc_info())
    return
        
    
def TwitterStreaming(company_names):
    ## twitter authentication keys
    
      

    consumer_key        = "yoWOau00G19Q81WKeVZ6g60zU"
    consumer_secret     = "A0rJ4XlMndHv2xTeQlA2t7N9thBr3FDRu6vkrCy5ab7KAiKmNB"
    access_token        = "16859687-bt1jbTlHUXO39n114gWEpg24VlKZQVbaF4AgXs4ha"
    access_token_secret = "kLMge9f3GypNwHv6N9uMCuUdLS7kr5gfR5lzTXEmwMyfi"
    global keywords
    keywords = [company_names]
    try:
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth)
        
        listener = TwitterListener(api, "test")
        print ("Begin Twitter streaming for ", keywords)
        stream = tweepy.Stream(auth, listener)
        print (keywords)
        stream.filter(track=company_names, follow="")
    except:
        print ("Error in Twitter streaming",sys.exc_info())
    return True

In [ ]:
company_keyword_count = 10
company_random = random.choice(company_names, company_keyword_count)
companieslist = ",".join("\""+str(companies.rstrip())+"\"" for companies in company_random)
print (companieslist)

companieslist = ['Procter & Gamble']

"Fidelity National Information Services","Costco","Huntington Bancshares","KeyCorp","Coach","Teradata Corp","American Tower Corp A","Emerson Electric","Prudential Financial","Macy's"


In [ ]:
TwitterStreaming(companieslist)